In [325]:
import itertools

import numpy as np
import pandas as pd

from operator import itemgetter
from decimal import Decimal


## General functions

In [326]:
def clean_integrals(Z, file_name):

    df = pd.read_csv(file_name, header=None)
    integrals_dict = {}

    for row in df.iterrows():

        key, val = row[1][0].split('=')

        key = key.replace('<', '')
        key = key.replace('>', '')
        key = key.replace('|V|', '')
        key = key.replace(' ', '')

        val = val.replace('[', '(')
        val = val.replace(']', ')')
        val = val.replace('Sqrt', 'np.sqrt')
        integrals_dict.update({f'{key}': eval(val)})

    return integrals_dict

In [327]:
hole_states = ['1+', '1-']
particle_states = ['2+', '2-', '3+', '3-']

def matrix_config(hole_states, particle_states):
    excitations = list(itertools.product(hole_states, particle_states))
    excitations = [list(x) for x in excitations if x[0][1]==x[1][1]]  ## removing non-zero-spin excitations
    excitations = sorted(excitations, key=itemgetter(1)) ## sorting by particle state

    bra = ket = [['0','0']] + excitations  ## ground state and excitations

    elements = list(itertools.product(bra, ket))
    elements = [list(element) for element in elements]

    matrix = np.zeros((5,5), dtype = object)        ## doing this because numpy reshape is acting weird
    row = 0
    for i, element in enumerate(elements):
        col = i % 5
        matrix[row, col] = element
        if col == 4 and i != 0:
            row += 1

    return matrix

In [328]:
def h0_term(alpha, beta, Z):
    if not alpha == beta:
        return 0

    n = int(alpha[0])
 
    return -(Z**2)/(2*n**2)

def v_term(alpha, beta, gamma, delta, integrals, antisym = True):

    alpha_n, alpha_spin = alpha[0], alpha[-1]
    beta_n, beta_spin = beta[0], beta[-1]
    gamma_n, gamma_spin = gamma[0], gamma[-1]
    delta_n, delta_spin = delta[0], delta[-1]

    direct_term = integrals[alpha_n+beta_n+gamma_n+delta_n]
    exchange_term = integrals[alpha_n+beta_n+delta_n+gamma_n]

    if alpha_spin != gamma_spin or beta_spin != delta_spin:
        direct_term = 0

    if alpha_spin != delta_spin or beta_spin != gamma_spin:
        exchange_term = 0

    if not antisym:
        return direct_term

    return direct_term - exchange_term

def below_fermi(fermi_level):
    n = [str(n) for n in range(1, fermi_level + 1)] 
    spin = ["+", "-"] 
    return  list(itertools.product(n, spin))


def zp_zh(Z, fermi_level, integrals): ## this is the E_0 ref
    cumul_h0 = 0
    cumul_v = 0

    states_below_fermi = below_fermi(fermi_level)

    #cumul_h0
    for i in states_below_fermi:
        cumul_h0 += h0_term(i, i, Z)

    #cumul_v
    for i in states_below_fermi:
        for j in states_below_fermi:
            cumul_v += v_term(i, j, i, j, integrals)

    return cumul_h0 + (1/2)*cumul_v

def onep_oneh(Z, fermi_level, a, i, b, j, integrals):
    cumul_h0 = 0
    cumul_v = 0
    states_below_fermi = below_fermi(fermi_level)

    if a==b and i==j and a[0]!= '0' :   ## diagonal
        cumul_h0 += h0_term(a, a, Z)
        for j in states_below_fermi:
            cumul_v += v_term(a, j, a, j, integrals)

        cumul_h0 += - h0_term(i, i, Z)
        for j in states_below_fermi:
            cumul_v += - v_term(i, j, i, j, integrals)

        return cumul_h0 + cumul_v + zp_zh(Z, fermi_level, integrals) ## this last term is E_0 ref
    
    # from core to excitation
    elif (a[0]==i[0]=='0') or (b[0]==j[0]=='0'): ## notice that the case where all are equal to zero will not happend because it is not passed to 1p1h

        if (b[0]=='0'): # than we change. we can do this because the matrix is symetric
            b, j = a, i
        for i in states_below_fermi:
            cumul_v +=  v_term(j, i, b, i, integrals) 

        return cumul_v

    else:
        return v_term(a, j, i, b, integrals) 

In [329]:
def jacobi(A, eps=1e-15, maxiter=10000):
    """
    Jacobi method for diagonalising a symmetric matrix A
    """
    n = A.shape[0]
    V = np.eye(n)
    for i in range(maxiter):
        # find the largest off-diagonal element
        maxval = 0
        for j in range(n):
            for k in range(j+1, n):
                if abs(A[j,k]) > maxval:
                    maxval = abs(A[j,k])
                    p = j # row index
                    q = k # column index
        if maxval < eps:
            break
        # rotate the matrix
        theta = 0.5 * np.arctan2(2*A[p,q], A[p,p]-A[q,q]) # rotation angle
        c = np.cos(theta)
        s = np.sin(theta)
        R = np.eye(n)
        R[p,p] = c
        R[p,q] = -s
        R[q,p] = s
        R[q,q] = c
        A = R.T @ A @ R # matrix rotation
        V = V @ R 
    return A, V # diagonalised matrix and eigenvectors

In [330]:
def matrix_parser(matrix, dimension):
    strin = ''
    for i in range(0, dimension):
        for j in range(0, dimension):
            strin += f"{matrix[i, j]: 6.3f} "

        strin += '\n'

    print(strin)

## Configuration Interaction

### Helium

In [331]:
Z = 2
fermi_level = 1
hole_states = ['1+', '1-']
particle_states = ['2+', '2-', '3+', '3-']

In [332]:
integrals = clean_integrals(Z, 'integrals.csv')

In [333]:
matrix = matrix_config(hole_states, particle_states)

for row in range(0,5):
    for col in range(0, 5):
        i = matrix[row, col][0][0]
        a = matrix[row, col][0][1] 
        j = matrix[row, col][1][0]
        b = matrix[row, col][1][1]
        if a == i == j == b== '0':    ## 0p0h 0p0h
            matrix[row, col] = zp_zh(Z, fermi_level, integrals)

        else:
            matrix[row, col] = onep_oneh(Z, fermi_level, a, i, b, j, integrals)

In [334]:
matrix_parser(matrix, 5)

-2.750  0.179  0.179  0.088  0.088 
 0.179 -1.704  0.044 -0.079  0.022 
 0.179  0.044 -1.704  0.022 -0.079 
 0.088 -0.079  0.022 -1.836  0.012 
 0.088  0.022 -0.079  0.012 -1.836 



In [335]:
diag_matrix = jacobi(matrix)[0] ## matrix with eigenvalues

In [336]:
matrix_parser(diag_matrix, 5)

-1.601  0.000  0.000  0.000 -0.000 
-0.000 -1.685 -0.000 -0.000 -0.000 
-0.000  0.000 -1.810  0.000 -0.000 
-0.000  0.000  0.000 -1.910 -0.000 
-0.000  0.000  0.000  0.000 -2.824 



### Beryllium

In [337]:
Z = 4
fermi_level = 2
hole_states = ['1+', '1-', '2+', '2-']
particle_states = ['3+', '3-']

In [338]:
integrals = clean_integrals(Z, 'integrals.csv')

In [339]:
matrix = matrix_config(hole_states, particle_states)

for row in range(0,5):
    for col in range(0, 5):
        i = matrix[row, col][0][0]
        a = matrix[row, col][0][1] 
        j = matrix[row, col][1][0]
        b = matrix[row, col][1][1]
        if a == i == j == b== '0':    ## <0p0h|0p0h>
            matrix[row, col] = zp_zh(Z, fermi_level, integrals)

        else:
            matrix[row, col] = onep_oneh(Z, fermi_level, a, i, b, j, integrals)

In [340]:
matrix_parser(matrix, 5)

-13.716  0.189  0.445  0.189  0.445 
 0.189 -9.280 -0.001  0.023  0.008 
 0.445 -0.001 -13.382  0.008  0.030 
 0.189  0.023  0.008 -9.280 -0.001 
 0.445  0.008  0.030 -0.001 -13.382 



In [341]:
diag_matrix = jacobi(matrix)[0] ## matrix with eigenvalues

In [342]:
matrix_parser(diag_matrix, 5)

-9.241  0.000 -0.000 -0.000 -0.000 
 0.000 -9.303 -0.000  0.000  0.000 
 0.000  0.000 -12.886 -0.000  0.000 
 0.000  0.000  0.000 -13.412  0.000 
 0.000  0.000  0.000  0.000 -14.198 



## Hartree-Fock

### Helium

In [343]:
Z = 2
hole_states = ['1+', '1-']
particle_states = ['2+', '2-', '3+', '3-'] ## separating hole and particle states here is useless but i do it to maintain the previous code structure

spOrbitals = hole_states + particle_states

Nparticles = Z ## incorect if ion but in our case it is ok

integrals = clean_integrals(Z, 'integrals.csv')

In [344]:
def set_up_density_matrix(Nparticles, spOrbitals):
    n_spOrbitals = len(spOrbitals)
    C = np.eye(n_spOrbitals) # HF coefficients
    DensityMatrix = np.zeros([n_spOrbitals,n_spOrbitals])
    for gamma in range(n_spOrbitals):
        for delta in range(n_spOrbitals):
            sum = 0.0
            for i in range(Nparticles):
                sum += C[gamma][i]*C[delta][i]
                DensityMatrix[gamma][delta] = Decimal(sum)

    return DensityMatrix

In [345]:
def hartee_fock(Nparticles, spOrbitals, integrals , maxHFiter= 100, epsilon = 1.0e-10, silent=False):
    DensityMatrix = set_up_density_matrix(Nparticles, spOrbitals)
    difference = 1.0
    hf_count = 0
    n_spOrbitals = len(spOrbitals)

    oldenergies = np.zeros(n_spOrbitals)
    newenergies = np.zeros(n_spOrbitals)

    while hf_count < maxHFiter and difference > epsilon:
        HFmatrix = np.zeros([n_spOrbitals, n_spOrbitals]) # reset the HF matrix		
        for alpha_index, alpha in enumerate(spOrbitals):
            for beta_index, beta in enumerate(spOrbitals):
                sumFockTerm = 0.0 ##Setting up the Fock matrix
                for gamma_index, gamma in enumerate(spOrbitals):
                    for delta_index, delta in enumerate(spOrbitals):
                        sumFockTerm += DensityMatrix[gamma_index][delta_index]*v_term(alpha, gamma, beta, delta, integrals)
                        HFmatrix[alpha_index][beta_index] = Decimal(sumFockTerm)

                        if beta_index == alpha_index:   ### Adding the one-body term
                            HFmatrix[alpha_index][alpha_index] += h0_term(alpha, beta, Z)
        
        #print(matrix_parser(HFmatrix, len(HFmatrix[0])))

        spenergies, C = np.linalg.eigh(HFmatrix)

        DensityMatrix = np.zeros([n_spOrbitals, n_spOrbitals]) ## Setting up new density matrix
        for gamma in range(n_spOrbitals):
            for delta in range(n_spOrbitals):
                sum = 0.0
                for i in range(Nparticles):
                    sum += C[gamma][i]*C[delta][i]
                DensityMatrix[gamma][delta] = Decimal(sum)
        newenergies = spenergies
        sum =0.0 
        for i in range(n_spOrbitals): ## difference between previous and new sp HF energies
            sum += (abs(newenergies[i]-oldenergies[i]))/n_spOrbitals
        difference = sum
        oldenergies = newenergies
        hf_count += 1

        if not silent:
            print(f"############### Iteration {hf_count} ###############")
            print("Single-particle energies")
            for i in range(n_spOrbitals):
                print('{0:4d}  {1:.4f}'.format(i, Decimal(oldenergies[i])))
        
    return DensityMatrix

In [346]:
def finding_E(Nparticles, spOrbitals, integrals, maxHFiter= 100):
    DensityMatrix = hartee_fock(Nparticles, spOrbitals, integrals, maxHFiter, silent=True)
    n_spOrbitals = len(spOrbitals)

    E = 0.0
    for alpha_index, alpha in enumerate(spOrbitals):
        for beta_index, beta in enumerate(spOrbitals):
            E += DensityMatrix[alpha_index][beta_index]*h0_term(alpha, beta, Z)
            for gamma_index, gamma in enumerate(spOrbitals):
                for delta_index, delta in enumerate(spOrbitals):
                    E += (1/2)*DensityMatrix[alpha_index][beta_index]*DensityMatrix[gamma_index][delta_index]*v_term(alpha, gamma, beta, delta, integrals)
    print(f"Energy: {E}")

In [350]:
hartee_fock(Nparticles, spOrbitals, integrals , maxHFiter= 100, epsilon = 1.0e-10)

############### Iteration 1 ###############
Single-particle energies
   0  -0.7833
   1  -0.7833
   2  0.0396
   3  0.0396
   4  0.4535
   5  0.4535
############### Iteration 2 ###############
Single-particle energies
   0  -0.8721
   1  -0.8721
   2  0.0396
   3  0.0396
   4  0.4441
   5  0.4441
############### Iteration 3 ###############
Single-particle energies
   0  -0.8865
   1  -0.8865
   2  0.0394
   3  0.0394
   4  0.4401
   5  0.4401
############### Iteration 4 ###############
Single-particle energies
   0  -0.8882
   1  -0.8882
   2  0.0394
   3  0.0394
   4  0.4396
   5  0.4396
############### Iteration 5 ###############
Single-particle energies
   0  -0.8884
   1  -0.8884
   2  0.0394
   3  0.0394
   4  0.4395
   5  0.4395
############### Iteration 6 ###############
Single-particle energies
   0  -0.8885
   1  -0.8885
   2  0.0394
   3  0.0394
   4  0.4395
   5  0.4395
############### Iteration 7 ###############
Single-particle energies
   0  -0.8885
   1  -0.8885
   2  0.0

array([[ 9.62549015e-01,  4.27667850e-17, -1.78554612e-01,
         4.75783952e-18, -6.45496653e-02,  2.60781705e-18],
       [ 4.27667850e-17,  9.62549015e-01, -9.71742503e-17,
        -1.78554612e-01,  1.31419080e-16, -6.45496653e-02],
       [-1.78554612e-01, -9.71742503e-17,  3.31222090e-02,
         1.56717692e-17,  1.19740816e-02,  5.50084673e-18],
       [ 4.75783952e-18, -1.78554612e-01,  1.56717692e-17,
         3.31222090e-02, -2.52295648e-17,  1.19740816e-02],
       [-6.45496653e-02,  1.31419080e-16,  1.19740816e-02,
        -2.52295648e-17,  4.32877623e-03, -9.18033163e-18],
       [ 2.60781705e-18, -6.45496653e-02,  5.50084673e-18,
         1.19740816e-02, -9.18033163e-18,  4.32877623e-03]])

In [351]:
finding_E(Nparticles, spOrbitals, integrals, maxHFiter= 100)

Energy: -2.8310960867850428


In [352]:
finding_E(Nparticles, spOrbitals, integrals, maxHFiter= 1)

Energy: -2.8291928003392073


### Beryllium

In [353]:
Z = 4
hole_states = ['1+', '1-', '2+', '2-',]
particle_states = ['3+', '3-']   ## separating hole and particle states here is useless but i do it to maintain the previous code structure

spOrbitals = hole_states + particle_states

Nparticles = Z ## incorect if ion but in our case it is ok

integrals = clean_integrals(Z, 'integrals.csv')

In [354]:
hartee_fock(Nparticles, spOrbitals, integrals ,maxHFiter= 100, epsilon = 1.0e-10)

############### Iteration 1 ###############
Single-particle energies
   0  -3.9507
   1  -3.9507
   2  -0.1040
   3  -0.1040
   4  0.8657
   5  0.8657
############### Iteration 2 ###############
Single-particle energies
   0  -4.6291
   1  -4.6291
   2  -0.2958
   3  -0.2958
   4  0.8254
   5  0.8254
############### Iteration 3 ###############
Single-particle energies
   0  -4.6778
   1  -4.6778
   2  -0.3039
   3  -0.3039
   4  0.8136
   5  0.8136
############### Iteration 4 ###############
Single-particle energies
   0  -4.6854
   1  -4.6854
   2  -0.3050
   3  -0.3050
   4  0.8115
   5  0.8115
############### Iteration 5 ###############
Single-particle energies
   0  -4.6867
   1  -4.6867
   2  -0.3052
   3  -0.3052
   4  0.8112
   5  0.8112
############### Iteration 6 ###############
Single-particle energies
   0  -4.6869
   1  -4.6869
   2  -0.3053
   3  -0.3053
   4  0.8111
   5  0.8111
############### Iteration 7 ###############
Single-particle energies
   0  -4.6870
   1  -4.68

array([[ 9.94592373e-01,  3.10755213e-18, -4.65659878e-02,
         4.80098275e-18, -5.66568043e-02,  1.52804744e-18],
       [ 3.10755213e-18,  9.94592373e-01, -9.29466194e-17,
        -4.65659878e-02, -2.79283703e-17, -5.66568043e-02],
       [-4.65659878e-02, -9.29466194e-17,  5.99012437e-01,
         2.82976767e-17, -4.87881283e-01, -1.03541307e-17],
       [ 4.80098275e-18, -4.65659878e-02,  2.82976767e-17,
         5.99012437e-01,  1.92987987e-17, -4.87881283e-01],
       [-5.66568043e-02, -2.79283703e-17, -4.87881283e-01,
         1.92987987e-17,  4.06395190e-01, -1.70761842e-17],
       [ 1.52804744e-18, -5.66568043e-02, -1.03541307e-17,
        -4.87881283e-01, -1.70761842e-17,  4.06395190e-01]])

In [355]:
finding_E(Nparticles, spOrbitals, integrals, maxHFiter= 100)

Energy: -14.508252442377183


In [356]:
finding_E(Nparticles, spOrbitals, integrals, maxHFiter= 1)

Energy: -14.499822866520386
